In [1]:
import os
import numpy as np

In [2]:
from lsst.utils import getPackageDir

In [3]:
twinklesDir = getPackageDir('twinkles')
runMetaDataFile = os.path.join(twinklesDir, 'data/run1_metadata_v6.csv')
#InputVisitsFile = os.path.join(twinklesDir, 'data/SelectedKrakenVisits.csv')

In [4]:
import pandas as pd

In [5]:
runMetaDataFile

'/Users/rbiswas/doc/projects/DESC/Twinkles/data/run1_metadata_v6.csv'

In [6]:
from desc.monitor.dbConnection import dbInterface

/Users/rbiswas/src/LSST/sims_catalogs_generation/python/lsst/sims/catalogs/generation/db/dbConnection.py:438: UserWarning: Duplicate object type id 25 specified: 
Output object ids may not be unique.
This may not be a problem if you do not want globally unique id values
  'want globally unique id values')
/Users/rbiswas/src/LSST/sims_catalogs_generation/python/lsst/sims/catalogs/generation/db/dbConnection.py:438: UserWarning: Duplicate object type id 40 specified: 
Output object ids may not be unique.
This may not be a problem if you do not want globally unique id values
  'want globally unique id values')


In [7]:
dbConn = dbInterface(database='DESC_Twinkles_Level_2',
                     host='127.0.0.1', port='3307',
                     driver='mysql')

In [8]:
# Find the tables on the database
tables = dbConn._dbo.get_table_names()
print(tables)

['CcdVisit', 'ForcedSource', 'Object']


In [9]:
# Find the columns in the table CcdVisit
colNames = dbConn._dbo.get_column_names('CcdVisit')
#print(colNames)

In [10]:
obsHistIDsSuccess = set(dbConn._dbo.execute_arbitrary('SELECT visitId from CcdVisit')['visitId'])
# print(obsHistIDsSuccess)

In [11]:
# From https://github.com/DarkEnergyScienceCollaboration/Twinkles/issues/229 provided by @tony-johnson
DMFailures = set([681808, 1417225, 1957025, 2166623])

In [12]:
# PhoSim Failures (See Run1CPUTimes.ipynb notebook for detail. This is part of @sethdigel 's investigation)
run1meta = pd.read_csv(runMetaDataFile,index_col=0,usecols=["obshistid","expmjd","filter","rotskypos","altitude",\
                                                "rawseeing","airmass","sunalt","moonalt","moonphase","dist2moon","cputime",\
                                                "runlimit", "hostname"])
phoSimFailedObsHistIDs = set(run1meta.query('runlimit==1').index.values) 

## Sanity Checks 

In [13]:
print('Total number of Observations = {}'.format(len(run1meta)))

Total number of Observations = 1227


In [14]:
InputVisits = set(run1meta.index.values)

In [15]:
failures = InputVisits -  obsHistIDsSuccess

#####  Number of failures should = number of inputs - number of success

In [16]:
assert len(failures) == len(InputVisits) -len(obsHistIDsSuccess)

#####  Failures consist of DM failures and phosim Failures

In [17]:
failures == phoSimFailedObsHistIDs.union(DMFailures)

True

In [18]:
failures - phoSimFailedObsHistIDs - DMFailures

set()

#####  Missing light curve points are all phosim failures

In [19]:
missingObs = set([709569, 726636, 732065])

In [20]:
missingObs.intersection(phoSimFailedObsHistIDs) == missingObs

True

In [23]:
run1meta.head()

,expmjd,filter,rotskypos,altitude,rawseeing,airmass,moonalt,moonphase,dist2moon,sunalt,cputime,hostname,runlimit
obshistid,,,,,,,,,,,,,
200,59580.12500,2,256.415741,70.843140,0.744120,1.058624,-36.291664,3.884229,124.437035,-30.809488,30202.13,3,0
220,59580.13672,1,256.719421,67.748306,0.869226,1.080465,-36.219700,3.838714,124.329948,-32.225323,25589.48,3,0
230,59580.14062,3,256.751892,66.023071,0.791241,1.094442,-36.095425,3.812869,124.268242,-32.939915,28681.54,3,0
250,59580.15234,4,256.638519,62.998943,0.928894,1.122339,-35.733200,3.767761,124.158920,-34.057011,40600.57,3,0
276,59580.16406,5,256.267395,59.199089,1.132846,1.164212,-35.021816,3.711396,124.019577,-35.202927,66684.06,3,0


In [24]:
run1meta.ix[[709569, 726636, 732065]]

,expmjd,filter,rotskypos,altitude,rawseeing,airmass,moonalt,moonphase,dist2moon,sunalt,cputime,hostname,runlimit
obshistid,,,,,,,,,,,,,
709569,60627.28906,1,256.526154,61.586487,0.789843,1.136965,10.604475,90.610908,52.176231,-30.269590,427379.28,4,1
726636,60654.07031,1,98.571197,72.598625,0.694619,1.047963,47.175110,66.521378,54.930840,-22.024441,427991.56,4,1
732065,60662.15234,1,255.388199,74.614113,0.453555,1.037172,13.664585,90.242516,87.642433,-34.176361,430691.41,4,1


In [28]:
obs = pd.read_csv(os.path.join(twinklesDir, 'data/SelectedKrakenVisits.csv'), index_col='obsHistID')

In [30]:
obs.ix[[709569, 726636, 732065]][['moonPhase', 'filtSkyBrightness']]

,moonPhase,filtSkyBrightness
obsHistID,,
709569,90.610909,19.287629
726636,66.521378,20.071838
732065,90.242514,19.732872
